#### Импорт библиотек и изучение датасета:

In [49]:
import pandas as pd
import numpy as np
from statsmodels.stats.proportion import proportions_ztest as ztest
import matplotlib.pyplot as plt
import random

raw_data = pd.read_csv('jewerly.csv')

In [7]:
raw_data.head(5)

,date,Variant,cnt(addToCart),sum(addToCartItems)
0,2019-09-01,90,56,110247
1,2019-09-01,10,52,163098
2,2019-09-02,10,59,93317
3,2019-09-02,90,56,26457
4,2019-09-03,90,54,56418


In [8]:
raw_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 4 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   date                 50 non-null     object
 1   Variant              50 non-null     int64 
 2   cnt(addToCart)       50 non-null     int64 
 3   sum(addToCartItems)  50 non-null     int64 
dtypes: int64(3), object(1)
memory usage: 1.7+ KB


#### Формулируем гипотезы:

При формулировании гипотезы и выборе метрики следует учесть следующие факторы:
* несмотря на то, что основная цель работы интернет-магазина заключается в увеличении прибыли, наличие калькулятора или таблицы не является основным фактором, который влияет на средний чек покупателя. Например, на покупку товара может влиять состав изделия, дизайн и т.д. 
* возможность узнать свой размер, очевидно, влияет на последующее добавление товара в корзину. 

Таким образом, целесообразно в качестве метрики выбрать соотношение количества добавления товара (при калькуляторе или таблице) и общего количества добавления товаров за день.

*Гипотезы:*

H0: p_calc = p_table \
H1 : p_calc > p_table

#### Подготовка данных:

In [31]:
### группировка по датам и вычисление общего числа добавления товаров в корзину за день
common = raw_data.groupby('date')['cnt(addToCart)'].sum().reset_index()
common.head()

,date,cnt(addToCart)
0,2019-09-01,108
1,2019-09-02,115
2,2019-09-03,100
3,2019-09-04,105
4,2019-09-05,131


In [58]:
### высчитываем количество "успехов" и общее количество "исходов" для случаев использования калькулятора
calc = pd.DataFrame({'count': raw_data[raw_data['Variant'] == 10]['cnt(addToCart)']}).set_index(np.arange(calc.shape[0]))
calc['common'] = common['cnt(addToCart)']
calc_success = calc['count'].sum()
calc_total = calc['common'].sum()

In [59]:
### высчитываем количество "успехов" и общее количество "исходов" для случаев использования таблицы
table = pd.DataFrame({'count': raw_data[raw_data['Variant'] == 90]['cnt(addToCart)']}).set_index(np.arange(table.shape[0]))
table['common'] = calc['common']
table_success = table['count'].sum()
table_total = table['common'].sum()

#### Выбор статистического критерия (подхода)

Будем считать, что распределение количества добавления товаров в корзину в генеральной совокупности имеет нормальное распределение. Используем критерий z-test

In [66]:
my_count = np.array([calc_success, table_success])
my_nobs = np.array([calc_total, table_total])

result = ztest(my_count, my_nobs, alternative = 'larger')
print('p-value: ', result[1])

p-value:  0.33167751348863594


#### Выводы:

Статистический вывод: на имеющихся данных, с уровнем значимости 5% (уровнем доверия 95%), есть основания принять H0.

Содержательный вывод: использование калькулятора не увеличивает количество добавления товаров по сравнению с использованием таблицы.